In [1]:
import mlflow
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
business = pd.read_json("/Users/rebekahzhou/Downloads/Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json", lines=True)
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [3]:
boba = business[business['categories'].str.contains('Bubble Tea', case=False, na=False)]
boba.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
555,bdfZdB2MTXlT6-RBjSIpQg,Pho Bistro,903 Embarcadero Del Norte,Isla Vista,CA,93117,34.412934,-119.855531,3.0,184,1,"{'RestaurantsDelivery': 'True', 'BikeParking':...","Food, Restaurants, Chinese, Bubble Tea, Vietna...","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
663,epGYbd0_OeSxsSkw-Kx5Rg,Hippie Hi,1814 Drew St,Clearwater,FL,33764,27.968452,-82.762159,4.0,16,0,"{'HasTV': 'True', 'RestaurantsReservations': '...","Vegan, Gluten-Free, Restaurants, Bubble Tea, S...","{'Monday': '10:0-20:0', 'Tuesday': '10:0-20:0'..."
1447,i9eMSNpOA8BfHGZrM-9sZg,Kung Fu Tea,1469 Brace Rd,Cherry Hill,NJ,08034,39.889260,-75.015852,4.5,10,1,"{'WiFi': 'u'free'', 'RestaurantsTakeOut': 'True'}","Bubble Tea, Juice Bars & Smoothies, Food, Coff...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
1874,-3-6BB10tIWNKGEF0Es2BA,The 81 Hong Kong Cafe,"625 E Wetmore Rd, Ste 109",Tucson,AZ,85705,32.288540,-110.963144,4.0,133,1,"{'Alcohol': 'u'none'', 'DogsAllowed': 'False',...","Coffee & Tea, Asian Fusion, Food, Bubble Tea, ...","{'Monday': '0:0-0:0', 'Wednesday': '10:0-21:0'..."


In [4]:
boba.head(1).hours

3    {'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...
Name: hours, dtype: object

In [5]:
def total_hours_open(hours_dict):
    total = 0
    if not isinstance(hours_dict, dict):
        return 0  # skip if it's None or not a dict
    for time in hours_dict.values():
        try:
            open_time, close_time = time.split('-')
            open_hour, open_min = map(int, open_time.split(':'))
            close_hour, close_min = map(int, close_time.split(':'))

            open_minutes = open_hour * 60 + open_min
            close_minutes = close_hour * 60 + close_min

            # Handle overnight hours (e.g., 22:00-2:00)
            if close_minutes < open_minutes:
                close_minutes += 24 * 60

            total += (close_minutes - open_minutes) / 60
        except:
            continue
    return total

boba['total_hours_per_week'] = boba['hours'].apply(total_hours_open)

/var/folders/_l/3hppw5yx6713b6p1n23s7gcc0000gn/T/ipykernel_36155/3586092106.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boba['total_hours_per_week'] = boba['hours'].apply(total_hours_open)


In [6]:
def latest_closing_time(hours_dict):
    latest = 0
    if not isinstance(hours_dict, dict):
        return None
    for time in hours_dict.values():
        try:
            close_hour, close_min = map(int, time.split('-')[1].split(':'))
            minutes = close_hour * 60 + close_min
            if minutes < 5 * 60:  # Treat closing times like 2:00am as next-day
                minutes += 24 * 60
            if minutes > latest:
                latest = minutes
        except:
            continue
    return latest / 60 if latest > 0 else None

boba['latest_close_hour'] = boba['hours'].apply(latest_closing_time)

/var/folders/_l/3hppw5yx6713b6p1n23s7gcc0000gn/T/ipykernel_36155/3336259327.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boba['latest_close_hour'] = boba['hours'].apply(latest_closing_time)


In [7]:
boba['open_7_days'] = boba['hours'].apply(lambda x: int(isinstance(x, dict) and len(x) == 7))

/var/folders/_l/3hppw5yx6713b6p1n23s7gcc0000gn/T/ipykernel_36155/179727427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boba['open_7_days'] = boba['hours'].apply(lambda x: int(isinstance(x, dict) and len(x) == 7))


In [8]:
def is_open_weekends(hours_dict):
    if not isinstance(hours_dict, dict):
        return 0
    return int('Saturday' in hours_dict and 'Sunday' in hours_dict)

boba['open_weekends'] = boba['hours'].apply(is_open_weekends)

/var/folders/_l/3hppw5yx6713b6p1n23s7gcc0000gn/T/ipykernel_36155/3947722182.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boba['open_weekends'] = boba['hours'].apply(is_open_weekends)


In [9]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for day in days:
    col_name = f'open_{day.lower()}'
    boba[col_name] = boba['hours'].apply(lambda x: int(isinstance(x, dict) and day in x))


/var/folders/_l/3hppw5yx6713b6p1n23s7gcc0000gn/T/ipykernel_36155/3133260717.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boba[col_name] = boba['hours'].apply(lambda x: int(isinstance(x, dict) and day in x))
/var/folders/_l/3hppw5yx6713b6p1n23s7gcc0000gn/T/ipykernel_36155/3133260717.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boba[col_name] = boba['hours'].apply(lambda x: int(isinstance(x, dict) and day in x))
/var/folders/_l/3hppw5yx6713b6p1n23s7gcc0000gn/T/ipykernel_36155/3133260717.py:5: Se

In [10]:
numerical_features = [
    'total_hours_per_week',
    'stars',
    'review_count',
    'latest_close_hour'
]

categorical_features = [
    'open_weekends',
    'open_7_days',
    'open_monday',
    'open_tuesday',
    'open_wednesday',
    'open_thursday',
    'open_friday',
    'open_saturday',
    'open_sunday'
]

In [11]:
boba[numerical_features].info()

<class 'pandas.core.frame.DataFrame'>
Index: 477 entries, 3 to 150261
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_hours_per_week  477 non-null    float64
 1   stars                 477 non-null    float64
 2   review_count          477 non-null    int64  
 3   latest_close_hour     435 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 18.6 KB


In [12]:
boba['latest_close_hour'] = boba.latest_close_hour.fillna(0)
boba['latest_close_hour']

/var/folders/_l/3hppw5yx6713b6p1n23s7gcc0000gn/T/ipykernel_36155/1178305934.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boba['latest_close_hour'] = boba.latest_close_hour.fillna(0)


3         21.0
555       22.0
663       20.0
1447      21.0
1874      24.0
          ... 
148099     0.0
149093    24.0
149269     0.0
149614    23.0
150261     0.0
Name: latest_close_hour, Length: 477, dtype: float64

In [13]:
boba[numerical_features].isna().sum().sum()

0

In [14]:
boba[categorical_features].info()

<class 'pandas.core.frame.DataFrame'>
Index: 477 entries, 3 to 150261
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   open_weekends   477 non-null    int64
 1   open_7_days     477 non-null    int64
 2   open_monday     477 non-null    int64
 3   open_tuesday    477 non-null    int64
 4   open_wednesday  477 non-null    int64
 5   open_thursday   477 non-null    int64
 6   open_friday     477 non-null    int64
 7   open_saturday   477 non-null    int64
 8   open_sunday     477 non-null    int64
dtypes: int64(9)
memory usage: 37.3 KB


In [15]:
boba['is_open'].count()

477

In [16]:
mlflow.set_tracking_uri('https://mlops-lab4-262317540491.us-west2.run.app')
mlflow.set_experiment('yelp-lab-2')

2025/04/17 13:30:02 INFO mlflow.tracking.fluent: Experiment with name 'yelp-lab-2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1744921802693, experiment_id='1', last_update_time=1744921802693, lifecycle_stage='active', name='yelp-lab-2', tags={}>

In [17]:
y = boba.is_open
X = boba[numerical_features + categorical_features]
dt = DecisionTreeClassifier(max_depth=4)
dt.fit(X, y)
accuracy_score(y, dt.predict(X))

0.8050314465408805

In [18]:
with mlflow.start_run():
    # log parameters and log metrics
    # parameters: hyperparameters
    # metrics: model performance metrics

    mlflow.set_tags({"Model":"decision-tree", "Train Data": "all-data"})

    tree_depth = 5
    dt = DecisionTreeClassifier(max_depth=tree_depth)
    dt.fit(X, y)
    acc = accuracy_score(y, dt.predict(X))

    mlflow.log_param("max_depth", tree_depth)
    mlflow.log_metric("accuracy", acc)

mlflow.end_run()

2025/04/17 13:30:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-kit-494 at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/1/runs/a26636b17cb64b10a6741220a1d63ea6.
2025/04/17 13:30:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/1.


In [19]:
from sklearn.ensemble import RandomForestClassifier

with mlflow.start_run():
    mlflow.set_tags({"Model":"random-forest", "Train Data": "all-data"})

    ntree = 1000
    mtry = 4

    mlflow.log_params({'n_estimators':ntree, 'max_features':mtry})

    rf = RandomForestClassifier(n_estimators = ntree, max_features = mtry, oob_score = True)
    rf.fit(X,y)
    acc = rf.oob_score_
    #acc = accuracy_score(y, rf.predict(X))
    mlflow.log_metric('accuracy', acc)

mlflow.end_run()

2025/04/17 13:30:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run bustling-skunk-656 at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/1/runs/a1e1d25c13de485f97c2d6391411a395.
2025/04/17 13:30:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/1.


In [20]:
ntrees = [20,40,60,80,100]
mtrys = [3,4,5]
for i in ntrees:
    for j in mtrys:
        with mlflow.start_run():
            mlflow.set_tags({"Model":"random-forest", "Train Data": "all-data"})

            mlflow.log_params({'n_estimators':i, 'max_features':j})

            rf = RandomForestClassifier(n_estimators = i, max_features = j, oob_score = True)
            rf.fit(X,y)
            acc = rf.oob_score_
            #acc = accuracy_score(y, rf.predict(X))
            mlflow.log_metric('accuracy', acc)
        mlflow.end_run()

2025/04/17 13:30:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run bold-stork-601 at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/1/runs/6703983e43c04a5083085975f63ce013.
2025/04/17 13:30:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/1.
2025/04/17 13:30:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run thoughtful-rat-14 at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/1/runs/08764ccf4ac44525b579367b958a3262.
2025/04/17 13:30:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/1.
2025/04/17 13:30:05 INFO mlflow.tracking._tracking_service.client: 🏃 View run painted-cow-486 at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/1/runs/9e700336ddd64c279901d14bbe22c619.
2025/04/17 13:30:05 INFO mlflow.tracking._tracking_service.client: 🧪 View experimen

In [21]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import mlflow
import mlflow.sklearn
import warnings
warnings.filterwarnings("ignore")

mlflow.set_experiment('yelp-lab-2-best-models-final')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

from collections import defaultdict

# Store best results for each type
best_models = {
    'rf': {'score': -1, 'params': None},
    'xgb': {'score': -1, 'params': None},
    'logreg': {'score': -1, 'params': None}
}

def objective(params):
    classifier_type = params['type']
    del params['type']

    with mlflow.start_run():
        if classifier_type == 'rf':
            clf = RandomForestClassifier(**params, random_state=42)
        elif classifier_type == 'xgb':
            clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', **params)
        elif classifier_type == 'logreg':
            clf = LogisticRegression(penalty='l1', solver='liblinear', **params)
        else:
            return {'loss': 1, 'status': STATUS_OK}

        acc = cross_val_score(clf, X_train, y_train, cv=3, scoring='accuracy').mean()


        train_df = pd.concat([X_train.reset_index(drop=True), pd.Series(y_train, name="target")], axis=1)
        test_df = pd.concat([X_test.reset_index(drop=True), pd.Series(y_test, name="target")], axis=1)
        
        # Save datasets
        train_df.to_csv("train_data.csv", index=False)
        test_df.to_csv("test_data.csv", index=False)

        # Log datasets
        mlflow.log_artifact("train_data.csv", artifact_path="data")
        mlflow.log_artifact("test_data.csv", artifact_path="data")

        # Log to MLflow
        mlflow.set_tag("Model", classifier_type)
        mlflow.log_params(params)
        mlflow.log_metric("accuracy", acc)

        # Track best per model type
        if acc > best_models[classifier_type]['score']:
            best_models[classifier_type]['score'] = acc
            best_models[classifier_type]['params'] = params.copy()

        return {'loss': -acc, 'status': STATUS_OK}


# Hyperparameter search space
search_space = hp.choice('classifier_type', [

    # Random Forest
    {
        'type': 'rf',
        'n_estimators': hp.randint('rf_n_estimators', 50, 300),
        'max_depth': hp.choice('rf_max_depth', [None, 5, 10, 20]),
        'max_features': hp.choice('rf_max_features', ['sqrt', 'log2']),
        'class_weight': hp.choice('rf_class_weight', [None, 'balanced']),
    },

    # Logistic Regression (Lasso)
    {
        'type': 'logreg',
        'C': hp.loguniform('logreg_C', -4, 2),  # inverse of regularization strength
        'fit_intercept': hp.choice('logreg_intercept', [True, False])
    },

    # XGBoost
    {
        'type': 'xgb',
        'n_estimators': hp.randint('xgb_n_estimators', 50, 300),
        'max_depth': hp.choice('xgb_max_depth', [3, 5, 7, 10]),
        'learning_rate': hp.loguniform('xgb_lr', -4, 0),  # ~0.018 to 1.0
        'subsample': hp.uniform('xgb_subsample', 0.6, 1.0),
        'colsample_bytree': hp.uniform('xgb_colsample', 0.6, 1.0),
    }
])

# Run optimization
trials = Trials()
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42)
)

final_models = {}

for model_type, info in best_models.items():
    if info['params'] is None:
        continue  # No good trial for this type

    print(f"Training best {model_type.upper()} with score: {info['score']:.4f}")

    if model_type == 'rf':
        final_models['rf'] = RandomForestClassifier(**info['params'], random_state=42).fit(X_train, y_train)
    elif model_type == 'xgb':
        final_models['xgb'] = XGBClassifier(use_label_encoder=False, eval_metric='logloss', **info['params']).fit(X_train, y_train)
    elif model_type == 'logreg':
        final_models['logreg'] = LogisticRegression(penalty='l1', solver='liblinear', **info['params']).fit(X_train, y_train)

from sklearn.metrics import accuracy_score

# Log final models to MLflow
for model_type, model in final_models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    with mlflow.start_run(run_name=f"final_{model_type}"):
        mlflow.set_tag("Model", f"{model_type}_final")
        mlflow.log_params(best_models[model_type]['params'])
        mlflow.log_metric("final_test_accuracy", acc)
        mlflow.sklearn.log_model(model, artifact_path=f"{model_type}_final_model")

2025/04/17 13:30:11 INFO mlflow.tracking.fluent: Experiment with name 'yelp-lab-2-best-models-final' does not exist. Creating a new experiment.


  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2025/04/17 13:30:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run secretive-lamb-583 at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2/runs/6523a93f3ee245bb9af3dc727658578f.

2025/04/17 13:30:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2.



  2%|▏         | 1/50 [00:02<01:41,  2.07s/trial, best loss: -0.7647058823529411]

2025/04/17 13:30:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run burly-croc-693 at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2/runs/c161d8a6c31941c7a3be9ae602b277c8.

2025/04/17 13:30:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2.



...

2025/04/17 13:31:07 INFO mlflow.tracking._tracking_service.client: 🏃 View run amusing-jay-631 at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2/runs/6e6d83c6e80d4f828c5c0e6efd9cb3e1.

2025/04/17 13:31:07 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2.



 98%|█████████▊| 49/50 [00:55<00:01,  1.07s/trial, best loss: -0.7703081232492996]

2025/04/17 13:31:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run omniscient-shoat-696 at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2/runs/0ff140e9bc65481cb9b9e4b05c0fd653.

2025/04/17 13:31:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2.



100%|██████████| 50/50 [00:56<00:00,  1.12s/trial, best loss: -0.7703081232492996]
Training best RF with score: 0.7647
Training best XGB with score: 0.7647
Training best LOGREG with score: 0.7703


2025/04/17 13:31:12 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/04/17 13:31:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run final_rf at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2/runs/3ed0ce46e19d49d4ab4568e27ba6d6a3.
2025/04/17 13:31:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2.
2025/04/17 13:31:15 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2025/04/17 13:31:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run final_xgb at: https://mlops-lab4-262317540491.us-west2.run.app/#/experiments/2/runs/92ded4232de54b4d9f792e01ccb266ee.
2025/04/17 13:31:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://mlops-lab4-262317

In [23]:
runid = 'cc022ed50c3d4e74bba4f59094e8d2e3'
mod_path = f'runs:/{runid}/logreg_final_model'
registered_model = mlflow.register_model(model_uri = mod_path, name = 'boba_logreg')

Registered model 'boba_logreg' already exists. Creating a new version of this model...
2025/04/17 13:32:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: boba_logreg, version 1
Created version '1' of model 'boba_logreg'.


In [24]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

client.transition_model_version_stage(
    name="boba_logreg",
    version=registered_model.version,
    stage="Staging"
)


<ModelVersion: aliases=[], creation_timestamp=1744921968834, current_stage='Staging', description='', last_updated_timestamp=1744921974081, name='boba_logreg', run_id='cc022ed50c3d4e74bba4f59094e8d2e3', run_link='', source='mlflow-artifacts:/2/cc022ed50c3d4e74bba4f59094e8d2e3/artifacts/logreg_final_model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [25]:
logged_model = 'models:/boba_logreg/Staging' #replace with one of your models

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
sklearn_model = mlflow.sklearn.load_model(logged_model)
sklearn_model

LogisticRegression(C=1.4028648574289304, penalty='l1', solver='liblinear')